/home/a/anaconda3/envs/graph/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
import torch
import numpy as np


from data_loading import parse_data, make_binary


class Data:
    def __init__(self, x, y_train, y_test, edge_index) -> None:
        self.x = x.float()
        self.edge_index = edge_index.int()
        self.indices = indices
        self.y_train = y_train.float()
        self.y_test = y_test.float()
        
    def to(self, device):
        self.x = self.x.to(device)
        self.y_train = self.y_train.to(device)
        self.y_test = self.y_test.to(device)
        self.indices = self.indices.to(device)
        self.edge_index = self.edge_index.to(device)
        return self

dataset = 'cora'
x_, edge_index_, y_ = parse_data(dataset)
num_classes = 1
num_node_features = x_.shape[1]
indices, y_binary_train, y_binary_test = make_binary(y_, 3, 0.05)
data = Data(x_, y_binary_train, y_binary_test, edge_index_)

In [23]:
data.y_train.dtype

torch.float32

In [25]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GATConv
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T

import matplotlib.pyplot as plt

from sklearn.metrics import f1_score 

from loss import PULoss



# name_data = 'Cora'
# dataset = Planetoid(root= '', name = name_data)
# dataset.transform = T.NormalizeFeatures()


# print(f"Number of Classes in {name_data}:", num_classes)
# print(f"Number of Node Features in {name_data}:", num_node_features)
 

nnpu = False

class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        # 2개의 GAT layer를 쌓을 것이다.
        # ( 2번째 : multi-head attention 사용 )
        self.conv1 = GATConv(num_node_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        #-------------------------------------#
        # x : (2708, 1433)
        # edge_index : (2, 10556)
        #-------------------------------------#
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        #-------------------------------------#
        # x : (2708, 64)
        # 64 : hid 차원(8) x head 개수 (8)
        #-------------------------------------#
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        #-------------------------------------#
        # x : (2708, 7)
        #-------------------------------------#
        return torch.sigmoid(x)        
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = GAT().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)  

model.train()
for epoch in range(500):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    # print(out[data.train_mask].squeeze(1).dtype, data.y[data.train_mask].dtype)
    criterion = nn.BCELoss()
    criterion = PULoss(nnpu=nnpu)
    loss = criterion(out[data.indices].squeeze(1), data.y_train[data.indices])
    
    if epoch%100 == 0:
        print(loss.item())
    
    loss.backward()
    optimizer.step()

model.eval()
output = model(data)

pred = torch.where(output < 0.5, torch.tensor(0, device=device), 
            torch.tensor(1, device=device))

f1 = f1_score(data.y_test[data.indices].cpu(), pred[data.indices].cpu())
print(f1)

0.6843011975288391
0.6833291053771973
0.6637474894523621
0.6561630368232727
0.6273201107978821
0.7994579945799457


In [7]:
dataset = Planetoid(root= '', name = 'Cora')
dataset.num_features

1433

In [4]:
a = torch.tensor([[1], [2], [0]])
torch.sigmoid(a)
 

tensor([[0.7311],
        [0.8808],
        [0.5000]])

In [10]:
target = torch.empty(3).random_(2)
target.dtype

torch.float32

Epoch: 0, Loss: 0.6768154501914978, F1: 0.0, Accuracy: 0.6979320531757754
Epoch: 1, Loss: 0.534628689289093, F1: 0.0, Accuracy: 0.6979320531757754
Epoch: 2, Loss: 0.45098885893821716, F1: 0.0, Accuracy: 0.6979320531757754
Epoch: 3, Loss: 0.41454342007637024, F1: 0.0, Accuracy: 0.6979320531757754
Epoch: 4, Loss: 0.3937392234802246, F1: 0.004878048780487805, Accuracy: 0.6986706056129985
Epoch: 5, Loss: 0.38407644629478455, F1: 0.00730816077953715, Accuracy: 0.69903988183161
Epoch: 6, Loss: 0.3590630292892456, F1: 0.012150668286755772, Accuracy: 0.6997784342688331
Epoch: 7, Loss: 0.336917906999588, F1: 0.03601440576230493, Accuracy: 0.7034711964549483
Epoch: 8, Loss: 0.3060283660888672, F1: 0.08430913348946135, Accuracy: 0.7112259970457903
Epoch: 9, Loss: 0.2842823266983032, F1: 0.2272727272727273, Accuracy: 0.7363367799113737
Epoch: 10, Loss: 0.2638320028781891, F1: 0.3980487804878049, Accuracy: 0.7721565731166913
Epoch: 11, Loss: 0.24968835711479187, F1: 0.5871244635193132, Accuracy: 0.

In [2]:
target = data.y_train

positive, unlabelled = target == 1, target == 0
loss_func = nn.BCELoss(reduction='none')
out = model(data)
y_pos = loss_func(out.squeeze(1), torch.ones_like(target)) 


In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='../data', name="Cora")
data = dataset[0]
class_freq = data.y.bincount().float()
max_class = class_freq.argmax() 

tensor([0., 1.], device='cuda:0')

In [26]:
# read log.txt 

import pandas as pd
import matplotlib.pyplot as plt


known_table = {}

known_table["GCN+NRE"] = {
    "CiteSeer": {"f1": "66.2 +/- 1.1", "acc": "93.2 +/- 0.2"},
    "Cora": {"f1": "76.7 +/- 0.9", "acc": "92.7 +/- 0.2"}

}

known_table["GCN+URE"] = {
    "CiteSeer": {"f1": "42.6 +/- 1.7", "acc": "90.9 +/- 0.2"},
     "Cora": {"f1": "50.9 +/- 0.8", "acc": "88.0 +/- 0.1"}
}

# dict to dataframe
pd.DataFrame.from_dict({(i,j): known_table[i][j] 
                           for i in known_table.keys() 
                           for j in known_table[i].keys()},
                       orient='index')



f1           acc
GCN+NRE CiteSeer  66.2 +/- 1.1  93.2 +/- 0.2
        Cora      76.7 +/- 0.9  92.7 +/- 0.2
GCN+URE CiteSeer  42.6 +/- 1.7  90.9 +/- 0.2
        Cora      50.9 +/- 0.8  88.0 +/- 0.1

In [28]:
import os
file_list = os.listdir('logs')
rows = []

my_table = {}


model = "my_GCN"
for file_name in file_list:
    file_name = 'logs/' + file_name
    df = pd.read_csv(file_name, sep=' ', header=None)
    dataset = file_name.split('_')[0].split('/')[1]
    nnpu = file_name.split('_')[2][:-4]
    
    if nnpu == 'True':
        nnpu = "NRE"
    else:
        nnpu = "URE"

    method  = "{}+{}".format(model, nnpu)

    result_f1 = "{} +/- {}".format(round(df[0].mean() * 100, 1), round(df[0].std() * 100, 1))
    result_acc = "{} +/- {}".format(round(df[1].mean() * 100, 1), round(df[1].std() * 100, 1))

    if method not in my_table:
        my_table[method] = {
            dataset: {"f1": result_f1, "acc": result_acc}
        }
    else:
        my_table[method][dataset] = {"f1": result_f1, "acc": result_acc}

# sort the dictory by key
my_table["my_GCN+NRE"] = {k: my_table["my_GCN+NRE"][k] for k in sorted(my_table["my_GCN+NRE"].keys())}
print(my_table)

# add my_table to known_table
known_table.update(my_table)

pd.DataFrame.from_dict({(i, j): known_table[i][j] 
                           for i in known_table.keys() 
                           for j in known_table[i].keys()},
                       orient='index')

{'my_GCN+NRE': {'CiteSeer': {'f1': '65.7 +/- 0.7', 'acc': '89.0 +/- 0.2'}, 'Cora': {'f1': '76.2 +/- 0.6', 'acc': '88.2 +/- 0.3'}}, 'my_GCN+URE': {'CiteSeer': {'f1': '42.6 +/- 2.0', 'acc': '84.5 +/- 0.3'}, 'Cora': {'f1': '50.9 +/- 0.6', 'acc': '80.0 +/- 0.2'}}}


f1           acc
GCN+NRE    CiteSeer  66.2 +/- 1.1  93.2 +/- 0.2
           Cora      76.7 +/- 0.9  92.7 +/- 0.2
GCN+URE    CiteSeer  42.6 +/- 1.7  90.9 +/- 0.2
           Cora      50.9 +/- 0.8  88.0 +/- 0.1
my_GCN+NRE CiteSeer  65.7 +/- 0.7  89.0 +/- 0.2
           Cora      76.2 +/- 0.6  88.2 +/- 0.3
my_GCN+URE CiteSeer  42.6 +/- 2.0  84.5 +/- 0.3
           Cora      50.9 +/- 0.6  80.0 +/- 0.2

In [30]:
l_new = float("inf")
prior_new = 0

while True:
    l, prior = l_new, prior_new
    B = LBP(prior, A, P, U)
    

float